In [1]:
from tqdm import tqdm

import pandas as pd
import numpy as np

from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.document_loaders import UnstructuredExcelLoader
from langchain_community.document_loaders import DataFrameLoader
from langchain.retrievers import BM25Retriever, EnsembleRetriever

In [2]:
class CFG:
    # store="프랭크버거"
    output_path = "/home/user12/beaver/data/db"
    save_path = ""
    embedding_model="BAAI/bge-m3"
    retriever_k=5
    retriever_bert_weight=0.7
    version='9'
    seed=42
    
# CFG.save_path = CFG.output_path + CFG.store + "_temp.csv"

In [ ]:
data_df = pd.read_excel(f'/home/user12/beaver/data/beaver_v{CFG.version}.xlsx').rename(columns = {
    '상점명' : 'store', 
    '이전 슬롯' : 'before_slot', 
    '현재 슬롯' : 'current_slot', 
    '이전 입력' : 'before_input', 
    '현재 입력' : 'current_input',
    '이전 응대' : 'before_response',
    '현재 응대' : 'current_response'
})

data_df['store'] = data_df['store'].apply(lambda x: x.split("_")[0])

AttributeError: 'float' object has no attribute 'split'

In [13]:
#### 엑셀파일 DB화(pickle파일로 변환) ####
store = "홍콩반점"

from langchain.schema import Document

# df_2 = pd.read_excel(f'/home/user09/beaver/data/dataset_v{CFG.version}.xlsx')
loader = DataFrameLoader(df_2, page_content_column="상품명")
docs = loader.load()
df_2 = pd.read_excel(f'/home/user09/beaver/data/shared_files/dataset/dataset_v{CFG.version}.xlsx')

# 상품명, 종류, 옵션, 가격을 하나의 문서로 결합하여 저장
docs = [
    Document(
        page_content=row['상품명'],
        metadata={
            '종류': row['종류'],
            '옵션': row['옵션'],
            '가격': row['가격'],
            '특징': row['특징'],
        }
    )
    for _, row in df_2.iterrows()
]   
    
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.vectorstores import FAISS

encode_kwargs={'normalize_embeddings':True}
model_kwargs={'device':'cpu'}

hf = HuggingFaceBgeEmbeddings(
    model_name=CFG.embedding_model,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
)

db = FAISS.from_documents(docs, hf)

db.save_local(f"{CFG.output_path}/{store}_faiss{CFG.version}")
import pickle 
with open(f"{CFG.output_path}/{store}_docs{CFG.version}.pkl", "wb") as f:
    pickle.dump(docs, f)
    
db = FAISS.load_local(f"/home/user09/beaver/data/db/{store}_faiss{CFG.version}", hf, allow_dangerous_deserialization=True)
    
retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={"k": CFG.retriever_k}
)

bm25_retriever = BM25Retriever.from_documents(docs)
bm25_retriever.k = CFG.retriever_k

ensemble_retriever = EnsembleRetriever(
    retrievers=[retriever, bm25_retriever],
    weights=[CFG.retriever_bert_weight, 1 - CFG.retriever_bert_weight],
)

In [6]:
#### 엑셀파일 DB화(pickle파일로 변환) ####
store = "홍콩반점"

import pandas as pd
from langchain.schema import Document
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever

# 엑셀 파일 로드
df_2 = pd.read_excel(f'/home/user12/beaver/data/shared_files/dataset/dataset_v{CFG.version}.xlsx')

# Document 생성: '상품명'만을 page_content로 사용
docs = [
    Document(
        page_content=row['상품명'],
        metadata={
            '옵션': row['옵션'],
            '특징': row['특징']
        }
    )
    for _, row in df_2.iterrows()
]

# Embeddings 설정
encode_kwargs={'normalize_embeddings':True}
model_kwargs={'device':'cpu'}

hf = HuggingFaceEmbeddings(
    model_name=CFG.embedding_model,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
)

# FAISS 벡터스토어 생성: 'page_content'만 사용하도록 text_field 설정
db = FAISS.from_documents(
    documents=docs,
    embedding=hf
)

# 벡터스토어 저장
db.save_local(f"{CFG.output_path}/{store}_faiss{CFG.version}")

# Document 리스트 저장
import pickle 
with open(f"{CFG.output_path}/{store}_docs{CFG.version}.pkl", "wb") as f:
    pickle.dump(docs, f)

# 벡터스토어 로드
db = FAISS.load_local(
    folder_path=f"/home/user12/beaver/data/db/{store}_faiss1",
    embeddings=hf,
    allow_dangerous_deserialization=True
)

# Retriever 설정: 'page_content'만 사용하도록 document_content 설정
retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={"k": CFG.retriever_k},
)

# BM25 Retriever 생성: 'page_content'만 사용하도록 content_field 설정
bm25_retriever = BM25Retriever.from_documents(
    docs,
)
bm25_retriever.k = CFG.retriever_k

# Ensemble Retriever 설정
ensemble_retriever = EnsembleRetriever(
    retrievers=[retriever, bm25_retriever],
    weights=[CFG.retriever_bert_weight, 1 - CFG.retriever_bert_weight],
)


/home/user12/venv/beaver/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [5]:
# store_list = ['홍콩반점']
# data_df___ = pd.DataFrame()
store = "홍콩반점"

from langchain.schema import Document
# df = pd.read_excel("/home/user12/beaver/홍콩반점_강남역점_매장정보.xlsx")

# df['OPTION_GROUP_NM'] = df['OPTION_GROUP_NM'].apply(lambda x: "음료 선택" if x=="음료선택" else x)


# df = df[['PROD_NM', 'CATEGORY_NM', 'OPTION_GROUP_NM', 'ESS_YN', 'OPTION_PROD_NM']]
# df = df.rename(columns = {
#         'PROD_NM' : '상품명',
#         'CATEGORY_NM' : '카테고리',
#         'OPTION_GROUP_NM' : '옵션그룹',
#         'ESS_YN' : '필수여부',
#         'OPTION_PROD_NM' : '옵션명'
#     })
# df = df.groupby(["상품명", "카테고리", "옵션그룹", "필수여부",])["옵션명"].apply(list).reset_index()


# menu_str = ""
# for name, df__ in df.groupby("카테고리"):
#     menu_str += f"- {name}: "
#     for i, df___ in df__.iterrows():
#         menu_str += f"{df___['상품명']}, "
#     menu_str += f"\n"
    
# data = {}
# for i, df__ in df.groupby('상품명'):
#     sub_data = {}
    
#     for j, df___ in df__.iterrows():
#         group = df___['옵션그룹']
#         ess = f"[필수 주문]" if df___['필수여부'] == 'Y' else "[선택 주문]" if df___['필수여부'] == 'N' else "None"
        
#         name = f"{ess} {group}"
#         sub_data[name] = df___['옵션명']
        
#     if ess == "None":
#         data[i] = "[옵션 없음]"
#     else:    
#         data[i] = str(sub_data)#.replace("'", "\"")
        
# df_2 = pd.read_excel(f'/home/user09/beaver/data/dataset_v{CFG.version}.xlsx')
# df_2 = df_2.reset_index()
# df_2.columns = ['상품명', '옵션']
# df_2.to_excel(f'/home/user09/beaver/data/dataset_v{CFG.version}.xlsx')

df_2 = pd.read_excel(f'/home/user12/beaver/data/dataset_v{CFG.version}.xlsx')
loader = DataFrameLoader(df_2, page_content_column="상품명")
docs = loader.load()
df_2 = pd.read_excel(f'/home/user12/beaver/data/dataset_v{CFG.version}.xlsx')

# 상품명, 종류, 옵션, 가격을 하나의 문서로 결합하여 저장
docs = [
    Document(
        page_content=row['상품명'],
        metadata={
            '종류': row['종류'],
            '옵션': row['옵션'],
            '가격': row['가격']
        }
    )
    for _, row in df_2.iterrows()
]   
    
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.vectorstores import FAISS

encode_kwargs={'normalize_embeddings':True}
model_kwargs={'device':'cpu'}

hf = HuggingFaceBgeEmbeddings(
    model_name=CFG.embedding_model,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
)

db = FAISS.from_documents(docs, hf)

db.save_local(f"{CFG.output_path}/{store}_faiss{CFG.version}")
import pickle 
with open(f"{CFG.output_path}/{store}_docs{CFG.version}.pkl", "wb") as f:
    pickle.dump(docs, f)
    
db = FAISS.load_local(f"/home/user12/beaver/data/db/{store}_faiss3", hf, allow_dangerous_deserialization=True)
    
retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={"k": CFG.retriever_k}
)

bm25_retriever = BM25Retriever.from_documents(docs)
bm25_retriever.k = CFG.retriever_k

ensemble_retriever = EnsembleRetriever(
    retrievers=[retriever, bm25_retriever],
    weights=[CFG.retriever_bert_weight, 1 - CFG.retriever_bert_weight],
)


/home/user12/venv/beaver/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [12]:
import pandas as pd
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.retrievers import BM25Retriever
from langchain.retrievers.ensemble import EnsembleRetriever
from langchain.schema import Document
import pickle

# 데이터 로드
df_2 = pd.read_excel(f'/home/user12/beaver/data/dataset_v{CFG.version}.xlsx')
loader = DataFrameLoader(df_2, page_content_column="상품명")
docs = loader.load()
df_2 = pd.read_excel(f'/home/user12/beaver/data/dataset_v{CFG.version}.xlsx')

store = '홍콩반점'

loader = DataFrameLoader(df_2, page_content_column="상품명")
docs = loader.load()
# 상품명, 종류, 옵션, 가격을 하나의 문서로 결합하여 저장
# docs = [
#     Document(
#         page_content=row['상품명'],
#         metadata={
#             '종류': row['종류'],
#             '옵션': row['옵션'],
#             '가격': row['가격']
#         }
#     )
#     for _, row in df_2.iterrows()
# ]

# HuggingFace 모델 설정
encode_kwargs = {'normalize_embeddings': True}
model_kwargs = {'device': 'cpu'}

hf = HuggingFaceBgeEmbeddings(
    model_name=CFG.embedding_model,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
)

# FAISS 인덱스 생성
db = FAISS.from_documents(docs, hf)

# FAISS 인덱스를 로컬에 저장
db.save_local(f"{CFG.output_path}/{store}_faiss{CFG.version}")

# docs를 피클로 저장
with open(f"{CFG.output_path}/{store}_docs{CFG.version}.pkl", "wb") as f:
    pickle.dump(docs, f)

# FAISS 인덱스 로드
db = FAISS.load_local(f"/home/user12/beaver/data/db/{store}_faiss{CFG.version}", hf, allow_dangerous_deserialization=True)

# 검색기 생성
retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={"k": CFG.retriever_k}
)

# BM25 검색기 생성
bm25_retriever = BM25Retriever.from_documents(docs)
bm25_retriever.k = CFG.retriever_k

# 앙상블 검색기 생성
ensemble_retriever = EnsembleRetriever(
    retrievers=[retriever, bm25_retriever],
    weights=[CFG.retriever_bert_weight, 1 - CFG.retriever_bert_weight],
)

# 검색 예시
# query = "고기짜장"  # 예시 쿼리
# results = ensemble_retriever.get_relevant_documents(query)
# print(results)
# print("------------------------------------------------")
# 결과 출력
# for result in results:
#     print(f"상품명: {result.page_content}")
#     print(f"종류: {result.metadata['종류']}")
#     print(f"옵션: {result.metadata['옵션']}")
#     print(f"가격: {result.metadata['가격']}")
#     print('-' * 50)


/home/user12/venv/beaver/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [6]:
data_df___.to_csv(f'/home/user12/beaver/testing_data/dataset_v{CFG.version}.csv', index=False)

In [7]:
from datasets import Dataset 
dataset = Dataset.from_pandas(data_df___)


from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=25, shuffle=True, random_state=CFG.seed)

for i, (train_index, val_index) in enumerate(skf.split(dataset['current_input'], dataset['store'])):
    train_dataset = dataset.select(train_index)
    val_dataset = dataset.select(val_index)
    
train_dataset.to_json(f"/home/user12/beaver/testing_data/dataset/beaver_v{CFG.version}_train.json")
val_dataset.to_json(f"/home/user12/beaver/testing_data/dataset/beaver_v{CFG.version}_val.json")

Creating json from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 339.59ba/s]


108718

In [8]:
import datasets
test_dataset = datasets.load_dataset('json', data_files=f'/home/user12/beaver/testing_data/dataset/beaver_v{CFG.version}_val.json')

Generating train split: 15 examples [00:00, 4246.97 examples/s]


In [9]:
test_dataset

DatasetDict({
    train: Dataset({
        features: ['store', 'before_slot', 'current_slot', 'before_input', 'before_response', 'current_input', 'current_response', 'retriever', 'all_menus'],
        num_rows: 15
    })
})

In [10]:
data_df__

,store,before_slot,current_slot,before_input,before_response,current_input,current_response,retriever,all_menus
0,프랭크버거,"{'매장포장여부': None, '주문내역': [None], '결제수단': None}","{'매장포장여부': None, '주문내역': [{'상품명': '베이컨 치즈 버거',...",이전 대화 없음,이전 대화 없음,베이컨 치즈 버거 단품 하나 주세요.,"네 ,베이컨 치즈 버거 단품 추가해드리겠습니다. 고객님! L/R 중에 사이즈를 선택...","- 베이컨 치즈 버거 세트: {'[필수 주문] 사이드 선택': ['콘 샐러드', '...","- 버거 단품: JG 버거, SG 불고기 버거, SG 크림 치즈 버거, 더블 치즈 ..."
1,프랭크버거,"{'매장포장여부': None, '주문내역': [{'상품명': '베이컨 치즈 버거',...","{'매장포장여부': None, '주문내역': [{'상품명': '베이컨 치즈 버거 세...",베이컨 치즈 버거 단품 하나 주세요.,"네 ,베이컨 치즈 버거 단품 추가해드리겠습니다. 고객님! L/R 중에 사이즈를 선택...",그냥 단품 말고 세트로 주세요.,"네 세트로 주문 도와드리겠습니다 고객님! 베이컨 치즈 버거 세트의 사이즈, 사이드,...","- 베이컨 치즈 버거 세트: {'[필수 주문] 사이드 선택': ['콘 샐러드', '...","- 버거 단품: JG 버거, SG 불고기 버거, SG 크림 치즈 버거, 더블 치즈 ..."
2,프랭크버거,"{'매장포장여부': None, '주문내역': [{'상품명': '베이컨 치즈 버거 세...","{'매장포장여부': None, '주문내역': [{'상품명': '베이컨 치즈 버거 세...",그냥 단품 말고 세트로 주세요.,"네 세트로 주문 도와드리겠습니다 고객님! 베이컨 치즈 버거 세트의 사이즈, 사이드,...",사이즈는 L로 부탁드리고요 사이드는 프렌치 프라이 음료는 아메리카노로 주세요,"네 고객님! L 사이즈에 프렌치 프라이, 아메리카노 옵션 추가해드렸습니다. ICE와...","- 베이컨 치즈 버거 세트: {'[필수 주문] 사이드 선택': ['콘 샐러드', '...","- 버거 단품: JG 버거, SG 불고기 버거, SG 크림 치즈 버거, 더블 치즈 ..."
3,프랭크버거,"{'매장포장여부': None, '주문내역': [{'상품명': '베이컨 치즈 버거 세...","{'매장포장여부': None, '주문내역': [{'상품명': '베이컨 치즈 버거 세...",사이즈는 L로 부탁드리고요 사이드는 프렌치 프라이 음료는 아메리카노로 주세요,"네 고객님! L 사이즈에 프렌치 프라이, 아메리카노 옵션 추가해드렸습니다. ICE와...",ICE로 주문 부탁드립니다.,네 ICE로 주문 도와드리겠습니다! 마지막으로 매장 포장여부와 결제수단을 말씀해주세요.,"- 베이컨 치즈 버거 세트: {'[필수 주문] 사이드 선택': ['콘 샐러드', '...","- 버거 단품: JG 버거, SG 불고기 버거, SG 크림 치즈 버거, 더블 치즈 ..."
4,프랭크버거,"{'매장포장여부': None, '주문내역': [{'상품명': '베이컨 치즈 버거 세...","{'매장포장여부': '포장', '주문내역': [{'상품명': '베이컨 치즈 버거 세...",ICE로 주문 부탁드립니다.,네 ICE로 주문 도와드리겠습니다! 마지막으로 매장 포장여부와 결제수단을 말씀해주세요.,포장해주세요,네 포장 주문 도와드리겠습니다! 결제 수단은 어떻게 하시겠어요?,"- 베이컨 치즈 버거 세트: {'[필수 주문] 사이드 선택': ['콘 샐러드', '...","- 버거 단품: JG 버거, SG 불고기 버거, SG 크림 치즈 버거, 더블 치즈 ..."
...,...,...,...,...,...,...,...,...,...
386,프랭크버거,"{'매장포장여부': None, '주문내역': [{'상품명': 'SG 불고기 버거 세...","{'매장포장여부': None, '주문내역': [{'상품명': 'SG 불고기 버거 세...",제로 사이다 없어요?,"죄송합니다 고객님. 저희 매장에는 제로 사이다가 없습니다. 저희 매장에는 콜라, 제...",그러면 제로 콜라라도 줘요,SG 크림 치즈 버거의 L 사이즈 옵션을 추가했습니다! 추가 메뉴를 주문하시려면 원...,"- SG 불고기 버거 세트: {'[필수 주문] 사이드 선택': ['콘 샐러드', '...","- 버거 단품: JG 버거, SG 불고기 버거, SG 크림 치즈 버거, 더블 치즈 ..."
387,프랭크버거,"{'매장포장여부': None, '주문내역': [{'상품명': 'SG 불고기 버거 세...","{'매장포장여부': None, '주문내역': [{'상품명': 'SG 불고기 버거 세...",그러면 제로 콜라라도 줘요,SG 크림 치즈 버거의 L 사이즈 옵션을 추가했습니다! 추가 메뉴를 주문하시려면 원...,치즈 프랜치 프라이 R 사이즈로 하나 더 줘요,치즈 프랜치 프라이를 추가했습니다! 추가 메뉴를 주문하시려면 원하시는 메뉴를 말씀해...,"- SG 불고기 버거 세트: {'[필수 주문] 사이드 선택': ['콘 샐러드', '...","- 버거 단품: JG 버거, SG 불고기 버거, SG 크림 치즈 버거, 더블 치즈 ..."
388,프랭크버거,"{'매장포장여부': None, '주문내역': [{'상품명': 'SG 불고기 버거 세...","{'매장포장여부': None, '주문내역': [{'상품명': 'SG 불고기 버거 세...",치즈 프랜치 프라이 R 사이즈로 하나 더 줘요,치즈 프랜치 프라이를 추가했습니다! 추가 메뉴를 주문하시려면 원하시는 메뉴를 말씀해...,됐어요,네 고객님 주문 도와드리겠습니다! 매장 포장여부와 결제수단을 말씀해주세요.,"- SG 불고기 버거 세트: {'[필수 주문] 사이드 선택': ['콘 샐러드', '...","- 버거 단품: JG 버거, SG 불고기 버거, SG 크림 치즈 버거, 더블 치즈 ..."
389,프랭크버거,"{'매장포장여부': None, '주문내역': [{'상품명': 'SG 불고기 버거 세...","{'매장포장여부': '포장', '주문내역': [{'상품명': 'SG 불고기 버거 세...",됐어요,네 고객님 주문 도와드리겠습니다! 매장 포장여부와 결제수단을 말씀해주세요.,포장이랑 카드로 해줘요,"고객님, 결제 전 주문 확인 부탁드리겠습니다! 이렇게 주문 도와드릴까요?<|show...","- SG 불고기 버거 세트: {'[필수 주문] 사이드 선택': ['콘 샐러드', '...","- 버거 단품: JG 버거, SG 불고기 버거, SG 크림 치즈 버거, 더블 치즈 ..."


In [19]:
import pandas as pd
import json

# Load the menu_info.json
menu_info_path = '/home/user12/beaver/data/dataset/menu_info.json'
with open(menu_info_path, 'r', encoding='utf-8') as f:
    menu_info = json.load(f)

# Check the loaded JSON
print(menu_info)

# Convert JSON to string for template
menu_info_json_str = json.dumps(menu_info, ensure_ascii=False)
print(menu_info_json_str)

{'버거 세트': ['JG 버거 세트', 'SG 불고기 버거 세트', 'SG 크림 치즈 버거 세트', '더블 치즈 버거 세트', '머쉬룸 버거 세트', '베이컨 치즈 버거 세트', '쉬림프 버거 세트', '치즈 버거 세트', '프랭크 버거 세트', '오딘에 반할라 버거 세트'], '버거 단품': ['JG 버거', 'SG 불고기 버거', 'SG 크림 치즈 버거', '더블 치즈 버거', '머쉬룸 버거', '베이컨 치즈 버거', '치즈 버거', '프랭크 버거', '쉬림프 버거'], '사이드': ['JG 치즈 스틱', '후라이드 아이스볼', '프렌치 프라이', '치즈 프렌치 프라이', '코올슬로 샐러드', '콘 샐러드', '피넛 슈가볼', '치킨', '스파이시 텐더', '더치즈볼', '통가슴 후라이드', '자이언트 통다리', '콘 치즈볼', '게다리살 튀김', '초코칩 쿠키', '치즈 쿠키'], '음료': ['콜라', '제로 콜라', '사이다', '오렌지 주스', '아메리카노', '밀크 쉐이크', '생수']}
{"버거 세트": ["JG 버거 세트", "SG 불고기 버거 세트", "SG 크림 치즈 버거 세트", "더블 치즈 버거 세트", "머쉬룸 버거 세트", "베이컨 치즈 버거 세트", "쉬림프 버거 세트", "치즈 버거 세트", "프랭크 버거 세트", "오딘에 반할라 버거 세트"], "버거 단품": ["JG 버거", "SG 불고기 버거", "SG 크림 치즈 버거", "더블 치즈 버거", "머쉬룸 버거", "베이컨 치즈 버거", "치즈 버거", "프랭크 버거", "쉬림프 버거"], "사이드": ["JG 치즈 스틱", "후라이드 아이스볼", "프렌치 프라이", "치즈 프렌치 프라이", "코올슬로 샐러드", "콘 샐러드", "피넛 슈가볼", "치킨", "스파이시 텐더", "더치즈볼", "통가슴 후라이드", "자이언트 통다리", "콘 치즈볼", "게다리살 튀김", "초코칩 쿠키", "치즈 쿠키"], "음료": ["콜라", "제로 콜라", "사이다", "오렌지 주스", "아

In [20]:
output_json_path = '/home/user12/beaver/data/dataset/menu_info_converted.json'
with open(output_json_path, 'w', encoding='utf-8') as json_file:
    json.dump(menu_info, json_file, ensure_ascii=False, indent=4)

print(f"JSON data saved to {output_json_path}")

JSON data saved to /home/user12/beaver/data/dataset/menu_info_converted.json


In [21]:
menu_info_path = '/home/user12/beaver/data/dataset/menu_info.json'
with open(menu_info_path, 'r', encoding='utf-8') as f:
    menu_info = json.load(f)

# Convert JSON to string for template
menu_info_json_str = json.dumps(menu_info, ensure_ascii=False)
print(menu_info_json_str)

# Save the JSON string to a file
output_json_path = '/home/user12/beaver/data/dataset/menu_info_converted.json'
try:
    with open(output_json_path, 'w', encoding='utf-8') as json_file:
        json_file.write(menu_info_json_str)
    print(f"JSON data saved to {output_json_path}")
except Exception as e:
    print(f"An error occurred: {e}")

{"버거 세트": ["JG 버거 세트", "SG 불고기 버거 세트", "SG 크림 치즈 버거 세트", "더블 치즈 버거 세트", "머쉬룸 버거 세트", "베이컨 치즈 버거 세트", "쉬림프 버거 세트", "치즈 버거 세트", "프랭크 버거 세트", "오딘에 반할라 버거 세트"], "버거 단품": ["JG 버거", "SG 불고기 버거", "SG 크림 치즈 버거", "더블 치즈 버거", "머쉬룸 버거", "베이컨 치즈 버거", "치즈 버거", "프랭크 버거", "쉬림프 버거"], "사이드": ["JG 치즈 스틱", "후라이드 아이스볼", "프렌치 프라이", "치즈 프렌치 프라이", "코올슬로 샐러드", "콘 샐러드", "피넛 슈가볼", "치킨", "스파이시 텐더", "더치즈볼", "통가슴 후라이드", "자이언트 통다리", "콘 치즈볼", "게다리살 튀김", "초코칩 쿠키", "치즈 쿠키"], "음료": ["콜라", "제로 콜라", "사이다", "오렌지 주스", "아메리카노", "밀크 쉐이크", "생수"]}
JSON data saved to /home/user12/beaver/data/dataset/menu_info_converted.json


In [11]:
pip install chardet

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.4/199.4 kB 1.7 MB/s eta 0:00:0000:0100:01m

[notice] A new release of pip is available: 24.1.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [12]:
import chardet

with open('/home/user12/beaver/testing_data/dataset_v8.4.csv', 'rb') as rawdata:
    result = chardet.detect(rawdata.read(10000))

# check what the character encoding might be
print(result)


{'encoding': 'utf-8', 'confidence': 0.99, 'language': ''}
